# Merging Datasets

<br>

### Imports

In [18]:
#exports
import json
import pandas as pd

import os
from typing import Any

In [2]:
from IPython.display import JSON

<br>

### User Inputs

In [16]:
raw_data_dir = '../data/raw'
source_meta_dir = '../data/source_meta'

<br>

### Output Definitions

Before we collate the data sources into a single dataset we want to first define the attribute sets we want to have in our outputs, for attributes that are not categorical we will specify `None`.

In [5]:
#exports
output_col_categories = {
    'fuel_type': [
        'gas',
        'coal',
        'wind',
        'solar',
        'oil',
        'hydro',
        'nuclear',
        'biomass',
        'other' # e.g. for batteries or aggregators
    ],
    'plant_type': [
        'ccgt',
        'ocgt',
        'coal',
        'onshore_wind',
        'offshore_wind',
        'floating_wind',
        'conc_solar',
        'pv_solar',
        'oil',
        'run_of_river',
        'pumped_storage',
        'nuclear',
        'aggregator',
        'battery',
        'biomass',
        'other' # ideally no plants should come under this, the preference is to create a new category
    ],
    'capacity': None,
}

JSON(output_col_categories)

<IPython.core.display.JSON object>

<br>

### Source Definitions

#### ESAIL

We'll start by loading the dataset in

In [6]:
filename = 'ESAIL.csv'

df_ESAIL = pd.read_csv(f'{raw_data_dir}/{filename}')

df_ESAIL.head()

,sett_bmu_id,ngc_bmu_id,bmu_root,name,primary_fuel_type,detailed_fuel_type,longitude,latitude
0,E_MARK-1,MARK-1,MARK,Rothes Bio-Plant CHP 1,biomass,bone,-3.603516,57.480403
1,E_MARK-2,MARK-2,MARK,Rothes Bio-Plant CHP 2,biomass,bone,-3.603516,57.480403
2,T_DIDC1,DIDC1,DIDC,Didcot A (G) 1,coal,coalgas_opt_out,-1.267570,51.623630
3,T_DIDC2,DIDC2,DIDC,Didcot A (G) 2,coal,coalgas_opt_out,-1.267570,51.623630
4,T_DIDC4,DIDC4,DIDC,Didcot A (G) 4,coal,coalgas_opt_out,-1.267570,51.623630


<br>

We'll then define the key column, as well as the mapping from the OSUKED key to the ESAIL key

In [7]:
key_col = 'sett_bmu_id'

key_map = (df_ESAIL
           [[key_col]]
           .reset_index()
           .pipe(lambda df: df.assign(index=df.index+10000))
           .set_index('index')
           [key_col]
           .to_dict()
          )

JSON([key_map])

<IPython.core.display.JSON object>

<br>

We'll also define how we want to extract data from the attribute columns, specifically we provide the name of the new output column, the rank of the source in regards to this column, and the value mapping necessary for that columns content to match the desired output.

In [79]:
attr_cols = {
    'primary_fuel_type': {
        'output_col': 'fuel_type',
        'output_rank': 0, # rank to determine which input to use when multiple are provided, 0 is highest
        'value_map': {
            'wind': 'wind', 
            'gas': 'gas', 
            'coal': 'coal', 
            'fuel_oil': 'oil', 
            'nuclear': 'nuclear', 
            'run_of_river': 'hydro',
            'pumped_storage': 'hydro', 
            'aggregator': 'other', 
            'other': 'other', 
            'rgt': 'gas', 
            'biomass': 'biomass', 
            'battery': 'other'
        }
    }
}

<br>

We can now combine this into a single metadata object for the source

In [14]:
ESAIL_def = {
    'filename': filename,
    'key_col': key_col,
    'key_map': key_map,
    'attr_cols': attr_cols
}

JSON(ESAIL_def)

<IPython.core.display.JSON object>

<br>

We'll save this source definition before moving on

In [17]:
with open(f'{source_meta_dir}/ESAIL.json', 'w') as f:
    json.dump(ESAIL_def, f)

<br>

#### GPPDB

In [ ]:
# 

<br>

### Constructing the Database

We'll begin by loading in and combining the definitions of each source

In [34]:
#exports
def load_source_definitions(source_meta_dir):
    source_definitions = dict()
    source_meta_filenames = [f for f in os.listdir(source_meta_dir) if '.json' in f]

    for filename in source_meta_filenames:
        source_name = filename.replace('.json', '')

        with open(f'{source_meta_dir}/{filename}', 'r') as f:
            source_definitions[source_name] = json.load(f)
            
    return source_definitions

In [35]:
source_definitions = load_source_definitions(source_meta_dir)
        
JSON(source_definitions)

<IPython.core.display.JSON object>

In [37]:
#exports
def identify_primary_keys(source_definitions):
    primary_keys = []

    for source in source_definitions.values():
        primary_keys += list(source['key_map'].keys())

    primary_keys = sorted(set(primary_keys))
    
    return primary_keys

In [38]:
primary_keys = identify_primary_keys(source_definitions)
    
primary_keys[:5]

['10000', '10001', '10002', '10003', '10004']

In [40]:
#exports
def check_source_for_disallowed_cols(attr_cols, output_col_categories):
    output_cols_to_be_added = [attr_col['output_col'] for attr_col in attr_cols.values()]
    output_cols_allowed = output_col_categories.keys()

    disallowed_output_cols = list(set(output_cols_to_be_added) - set(output_cols_allowed))

    assert len(disallowed_output_cols)==0, f"The following columns are not allowed in the output dataset: {', '.join(disallowed_output_cols)}"
    
def check_sources_for_disallowed_cols(source_definitions, output_col_categories):
    for source, definition in source_definitions.items():
        check_source_for_disallowed_cols(definition['attr_cols'], output_col_categories)
        
    return

In [41]:
check_sources_for_disallowed_cols(source_definitions, output_col_categories)

In [80]:
"""
Start this from scratch, also need the input col names
"""

#exports
def identify_output_cols_used(source_definitions):
    output_cols = []
    output_ranks = []
    output_sources = []

    for source, definition in source_definitions.items():
        # need source, col and rank for each, then return mapping from output col to ranked sources
        output_cols += [col_def['output_col'] for col_def in definition['attr_cols'].values()]
        output_ranks += [col_def['output_rank'] for col_def in definition['attr_cols'].values()]
        output_sources = [source] * len(output_ranks)
        
    df_output_cols = pd.DataFrame({
        'output_col': output_cols,
        'source_rank': output_ranks,
        'source': output_sources
    })

    return df_output_cols

In [81]:
output_cols = identify_output_cols_used(source_definitions)

output_cols

KeyError: 'output_rank'

In [71]:
#exports
def identify_output_col_sources(output_col, source_definitions):
    output_col_sources = list()
    
    for source, definition in source_definitions.items():
        source_output_cols = [col_def['output_col'] for col_def in definition['attr_cols'].values()]
        
        if output_col in source_output_cols:
            output_col_sources += [source]

    return output_col_sources

In [72]:
output_col = 'fuel_type'

output_col_sources = identify_output_col_sources(output_col, source_definitions)

output_col_sources

['ESAIL']

In [73]:
#exports
def identify_output_cols_sources(source_definitions):
    output_col_to_sources = dict()
    output_cols = identify_output_cols_used(source_definitions)

    for output_col in output_cols:
        output_col_sources = identify_output_col_sources(output_col, source_definitions)
        output_col_to_sources[output_col] = output_col_sources
        
    return output_col_to_sources

In [74]:
output_col_to_sources = identify_output_cols_sources(source_definitions)

output_col_to_sources

{'fuel_type': ['ESAIL']}

In [32]:
df = pd.DataFrame(index=primary_keys)
df.index.name = 'osuked_id'

df.head()

""
osuked_id
10000
10001
10002
10003
10004


In [42]:
# check that each source's input_key is unique
# determine output columns that will be included
# iterate over output cols not sources
# check that column rankings are unique
# start with lowest rank then work up